# 特徴量圧縮，学習モデル適用

以後，訓練データ，検証データをまとめてギャラリーデータとする．また，テストデータにおけるトラックレット毎のサンプルの塊をそれぞれクエリとする．

OSNetで得た外観特徴量を学習モデルに適用して，特徴量を圧縮する．以下のコードをinput_fileにギャラリーデータとテストデータをそれぞれ与えて，コードを実行する．

# 出力csv
- label
- point_ID
- file_name
- 圧縮後特徴量ベクトル128次元

In [ ]:
#入力データに512次元の特徴ベクトルとした場合
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# モデルの定義
class Net128(nn.Module):
    def __init__(self):
        super(Net128, self).__init__()
        self.fc1 = nn.Linear(512, 384)
        self.fc2 = nn.Linear(384, 288)
        self.bn1 = nn.BatchNorm1d(288)
        self.fc3 = nn.Linear(288, 224)
        self.dropout_1 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(224, 160)
        self.dropout_2 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(160, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc6 = nn.Linear(128, 128)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.bn1(x)
        x = F.relu(self.fc3(x))
        x = self.dropout_1(x)
        x = F.relu(self.fc4(x))
        x = self.dropout_2(x)
        x = F.relu(self.fc5(x))
        x = self.bn2(x)
        x = self.fc6(x)
        return x

def normalize_features(features):
    return features.apply(lambda x: (x - x.min()) / (x.max() - x.min()), axis=1)

def transform_features(model, input_csv_path, output_csv_path):
    # モデルのインスタンスを作成し、状態をロード
    model.eval()  # 評価モードに設定

    # CSVファイルからデータを読み込む
    df = pd.read_csv(input_csv_path)

    # 512次元の特徴量を取得
    features = df.iloc[:, 3:-4]
    features = normalize_features(features)
    features_tensor = torch.tensor(features.values, dtype=torch.float32)


    # 低次元に変換
    with torch.no_grad():
        transformed_features = model(features_tensor)

    # 結果をデータフレームに追加
    for i in range(transformed_features.shape[1]):
        column_name = f'dim{i + 1}'
        df[column_name] = transformed_features[:, i].numpy()

    # 3列目から514列目までを削除する（インデックス3から515）
    df = df.drop(df.columns[3:515], axis=1)

    # CSVとして保存
    df.to_csv(output_csv_path, index=False)

if __name__ == "__main__":
    model = Net128()  # モデルのインスタンス化
    model.load_state_dict(torch.load("./best_model_PML128_exp1_kouzumi.pth"))  # モデルの状態をロード
    input_file = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_with_exp1_koizumi.csv'
    output_csv_path = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/tranceformed_data_all_orient5_num_128_triplet_OSNet_with_exp1_koizumi.csv'

    transform_features(model, input_file, output_csv_path)